In [39]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import torch.nn.functional as F
import torch
import torchvision


In [40]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('Dig-MNIST.csv')

In [41]:
train_data=df_train.drop('label',axis=1)
train_targets=df_train['label']

X_test=df_test.drop('label',axis=1)
Y_test=df_test['label']

In [42]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data,
                                                  train_targets,
                                                  test_size=0.2)

In [43]:
X_train.reset_index(drop=True, inplace=True)
Y_train.reset_index(drop=True, inplace=True)

X_val.reset_index(drop=True, inplace=True)
Y_val.reset_index(drop=True, inplace=True)

X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)



In [44]:
IMGSIZE = 28

class KannadaDataSet(Dataset):
    def __init__(self, images, labels, transforms = None):
        self.X = images
        self.y = labels
        self.transforms = transforms
        
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X.iloc[i,:]
        data = np.array(data).astype(np.uint8).reshape(IMGSIZE,IMGSIZE,1)
        
        if self.transforms:
            data = self.transforms(data)

        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

In [45]:
train_trans = transforms.Compose(([transforms.ToPILImage(),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,))]))

val_trans = transforms.Compose(([transforms.ToPILImage(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))]))

In [46]:
batch_size = 128

train_data = KannadaDataSet(X_train, Y_train, train_trans)
val_data = KannadaDataSet(X_val, Y_val, val_trans)
test_data = KannadaDataSet(X_test, Y_test, val_trans)


train_loader = torch.utils.data.DataLoader(train_data, 
                                           batch_size=batch_size, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(val_data, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size, 
                                          shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1, padding=2)
        self.conv1_bn = nn.BatchNorm2d(num_features=4)

        self.conv1_1 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=4, stride=1, padding=1)
        self.conv1_1_bn = nn.BatchNorm2d(num_features=8)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2_bn = nn.BatchNorm2d(num_features=16)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.conv3_bn = nn.BatchNorm2d(num_features=32)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(32*2*2, 64)
        self.fc2 = nn.Linear(in_features=64, out_features=32) 
        self.fc3 = nn.Linear(in_features=32, out_features=16) 
        self.out = nn.Linear(in_features=16, out_features=10) 
        
    def forward(self, x):

        x = self.conv1(x)
        x = self.conv1_bn(x)
        x = F.relu(x)
        x = self.conv1_1(x)
        x = self.conv1_1_bn(x)
        x = F.relu(x)       
        
        x = self.pool1(x) 

        x = self.conv2(x)
        x = self.conv2_bn(x)
        x = F.relu(x)
        x = self.pool2(x) 
        
        x = self.conv3(x)
        x = self.conv3_bn(x)
        x = F.relu(x)
        x = self.pool3(x) 
        
        x = x.view(-1, 32*2*2)

        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc3(x))
        
        x = self.out(x)
        return F.log_softmax(x, dim=-1)


net = Net()
net

Net(
  (conv1): Conv2d(1, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv1_bn): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_1): Conv2d(4, 8, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (conv1_1_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Li

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=.5*1e-2)


In [49]:
EPOCHS=10

nn_output=[]

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

for epoch in range(EPOCHS):
    epoch_loss = 0
    epoch_correct = 0
    net.train()
    #i=0
    for data in train_loader:
        """i+=1
        if i%100==0:
            print(i)"""
        X = data[0]
        y = data[1]
        
        net.zero_grad()  
        output = net(X)
        tloss = criterion(output, y) 
        tloss.backward()  
        optimizer.step()  
        
        epoch_loss += tloss.item()
        epoch_correct += get_num_correct(output, y)
        
    net.eval() 
    val_loss = 0
    val_correct = 0
    test_loss = 0
    test_correct = 0
    
    with torch.no_grad():
        for data in val_loader:
            X = data[0]
            y = data[1]
            
            preds = net(X)
            vloss = criterion(preds, y)
            
            val_correct += get_num_correct(preds, y)
            val_loss += vloss.item()
            
    tmp_nn_output = [epoch + 1,EPOCHS,
                     epoch_loss/len(train_loader.dataset),epoch_correct/len(train_loader.dataset)*100,
                     val_loss/len(val_loader.dataset), val_correct/len(val_loader.dataset)*100,
                     test_loss/len(test_loader.dataset), test_correct/len(test_loader.dataset)*100
                    ]
    nn_output.append(tmp_nn_output)
    
    print('Epoch [{}/{}] train loss: {:.6f} acc: {:.3f} - valid loss: {:.6f} acc: {:.3f}'
        .format(*tmp_nn_output))

Epoch [1/10] train loss: 0.001560 acc: 93.404 - valid loss: 0.000469 acc: 98.075
Epoch [2/10] train loss: 0.000346 acc: 98.654 - valid loss: 0.001325 acc: 94.442
Epoch [3/10] train loss: 0.000282 acc: 98.929 - valid loss: 0.000227 acc: 99.175
Epoch [4/10] train loss: 0.000219 acc: 99.148 - valid loss: 0.000191 acc: 99.308
Epoch [5/10] train loss: 0.000177 acc: 99.321 - valid loss: 0.000212 acc: 99.217
Epoch [6/10] train loss: 0.000173 acc: 99.296 - valid loss: 0.000334 acc: 98.700
Epoch [7/10] train loss: 0.000152 acc: 99.371 - valid loss: 0.000239 acc: 99.100
Epoch [8/10] train loss: 0.000139 acc: 99.456 - valid loss: 0.000200 acc: 99.408
Epoch [9/10] train loss: 0.000115 acc: 99.544 - valid loss: 0.000177 acc: 99.417
Epoch [10/10] train loss: 0.000121 acc: 99.575 - valid loss: 0.000213 acc: 99.283


In [50]:
  with torch.no_grad():
        for data in test_loader:
            X = data[0]
            y = data[1]
            
            preds = net(X) 
            tstloss = criterion(preds, y) 
            
            test_correct += get_num_correct(preds, y)
            test_loss += tstloss.item()
print(test_loss)

94.3877487629652
